In [114]:
%load_ext autoreload
%autoreload 2

## Import necessary modules
import os,sys
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator, AutoLocator, MultipleLocator, FormatStrFormatter, ScalarFormatter, FuncFormatter
import numpy as np
import datetime, calendar
from datetime import timedelta
import matplotlib.patches as mpatches
from itertools import tee
from traitlets import traitlets
import matplotlib.gridspec as gridspec

sys.path.append(os.path.abspath('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/entwuerfe/pyplots/lib/'))
from ce_funclib import determine_kernzeit as dtkz
from ce_funclib import continuity_check, decminutes_to_mmss as dec2min

from ipywidgets import widgets, interact, interactive, fixed, interact_manual, Layout
from IPython.display import display

%matplotlib tk

## Import data frome pickle generated from muß ein file mit agentenstats sein, wird generiert von Datensammler_Agenten.ipynb
pklpth='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/datenhalde/datapickles/Rohdaten_Agenten_aktuell.pkl'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Funktionen

In [115]:
def add_averages(frame):

    frame['avtt']=frame['tt_float']/frame['be']
    frame['avht']=frame['ht_float']/frame['be']
    frame['avacw']=frame['acw_float']/frame['be']
    
    colorder_with_abs=['an','be','vl','avht','avtt','avacw','ht_float','tt_float','acw_float']
    frame_with_abs=frame[colorder_with_abs]
    
    #colorder_av_only=['an','be','vl','avht','avtt','avacw']
    #frame_only_av=frame[colorder_av_only].fillna(0).unstack('bz')  # die Bearbeitungszeit soll hoch in die Spalten
    
    return frame_with_abs.fillna(0)

In [203]:
def prepare_plotframe(frame,von_bis):
    zeitraum_roh=frame.loc[(frame.index.date >= von_bis[0]) & (frame.index.date <= von_bis[1])].copy()
    von_bis_real=(zeitraum_roh.index.date.min(),zeitraum_roh.index.date.max())

    zeitraum_grp=zeitraum_roh.groupby([pd.Grouper('agent'),pd.Grouper('bz')]).sum().unstack('bz',fill_value=0).stack('bz')
    g = zeitraum_grp.sum(level=['agent']) # einen Zwischenframe erstellen, der nur die Kern- und Nebenzeit summiert
    zeitraum_grp=zeitraum_grp.append(g.assign(bz='g').set_index('bz', append=True)).sort_index() # die Summe dem alten Frame beigeben

    zeitraum_grp_plusavs=add_averages(zeitraum_grp)

    zindex=zeitraum_grp_plusavs.xs('g',level='bz').sort_values('avht').index   # nur gesamtzeit sortiert nach angenommenen, davon der index
    zeitraum_grp_plusavs=zeitraum_grp_plusavs.reindex(zindex,level=0) # frame danach sortiert
    
    return zeitraum_grp_plusavs,von_bis_real

### plot-funktionen

In [206]:
def agentplot(frame,vonbis):
    
    gcol,lostcol,verbcol,ncol,kcol,ncol2="#777777","#AC003A","#008EC4","#000000","#4F7DFF",'#424242'
    
    fig = plt.figure(figsize=(10,7))
    ax1 = plt.subplot2grid((1, 3), (0, 1), colspan=2)
    ax2 = plt.subplot2grid((1, 3), (0, 0), colspan=1)
    
    d_ix=frame.index.get_level_values(0).unique()
    agents_ticks=range(len(d_ix))
    
    ####################################################
    ### subplot für die Zeiten
    
    ## y-ticks nach Agenten
    ax1.set_yticks(agents_ticks)
    ax1.set_yticklabels(d_ix)
            
    ## locators und formatters für die rechte x-achse
    ax1.xaxis.set_major_locator(MultipleLocator(1))
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.xaxis.set_major_formatter(FuncFormatter(dec2min))
    ax1.xaxis.set_minor_formatter(FuncFormatter(dec2min))    # Die Dezimalzahlen der Y-Achse sollen labels im MM:SS Format erhalten
    
    for label in ax1.get_xticklabels(which='both'):          # lösung mit loop scheint am einfachsten zu sein...
        label.set_rotation(75) 
    for label in ax1.get_xticklabels(which='major'):
        label.set_fontsize(7)
        label.set_fontweight('bold')
    for label in ax1.get_xticklabels(which='minor'):
        label.set_fontsize(6)
    
    ## Hilfslinien
    ax1.grid(which='major', axis='x', linestyle='-', zorder=0, alpha=0.5)
    ax1.grid(which='minor', axis='x', linestyle='--', zorder=0, alpha=0.5)
    ax1.set_axisbelow(True)
    
    ## linien für mittel (median ist sinnlos, weil die ausgangsdaten schon nach Stunde zusammengefasst sind)
    meangcol,meankcol,meanncol = '#FF0000','#FBFF00','#42FF00'
    meanht_g=frame.xs('g', level='bz')['ht_float'].sum()/frame.xs('g', level='bz')['be'].sum()
    meanht_k=frame.xs('k', level='bz')['ht_float'].sum()/frame.xs('k', level='bz')['be'].sum()
    meanht_n=frame.xs('n', level='bz')['ht_float'].sum()/frame.xs('n', level='bz')['be'].sum()
    mean_line_g=ax1.axvline(x=meanht_g,color=meangcol,linewidth=1,label=str(dec2min(meanht_g))+' Ø-HT Kern+Neben Alle',zorder=0)
    mean_line_k=ax1.axvline(x=meanht_k,color=meankcol,linewidth=1,label=str(dec2min(meanht_k))+' Ø-HT Kern Alle',zorder=0)
    mean_line_n=ax1.axvline(x=meanht_n,color=meanncol,linewidth=1,label=str(dec2min(meanht_n))+' Ø-HT Neben Alle',zorder=0)
    
    ## barchart für die Durchschnittszeiten
    g_aht=frame.xs('g', level='bz')['avht'].values
    g_att=frame.xs('g', level='bz')['avtt'].values
    g_aacw=frame.xs('g', level='bz')['avacw'].values

    bar_aht=ax1.barh(agents_ticks,g_aht,color=gcol,height=0.8,label='gesamt_aht')
    bar_att=ax1.barh(agents_ticks,g_att,color=verbcol,height=0.2,label='ATT Kern+Neben (Agent)')
    bar_aacw=ax1.barh(agents_ticks,g_aacw,color=lostcol,left=g_att,height=0.2,label='AACW Kern+Neben (Agent)')
       
    #####################################################
    ### subplot für die zahl der calls
    summe_calls_gesamt=frame.xs('g', level='bz')['an'].sum()
    
    ## Y-Achse wie im ersten subplot, aber alles auf der rechten Seite und ohne labels
    ax2.set_yticks(ax1.get_yticks())
    ax2.set_ylim(ax1.get_ylim())
    ax2.yaxis.tick_right()
    ax2.tick_params(axis='y',which='both',labelright=False)
    
    ## locators und formatters für die X-Achse
    ax2.xaxis.set_major_locator(AutoLocator())
    ax2.xaxis.set_minor_locator(AutoMinorLocator())
    ax2.xaxis.set_minor_formatter(FormatStrFormatter("%d"))
    for label in ax2.get_xticklabels(which='both'):          # lösung mit loop scheint am einfachsten zu sein...
        label.set_rotation(75) 
    for label in ax2.get_xticklabels(which='major'):
        label.set_fontsize(7)
        label.set_fontweight('bold')
    for label in ax2.get_xticklabels(which='minor'):
        label.set_fontsize(6)

    
    ## barchart für angenommene calls
    
    kc_col,nc_col='#8D00CC','#51D5FF'
    
    g_calls = frame.xs('g', level='bz')['an'].values
    k_calls = frame.xs('k', level='bz')['an'].values
    n_calls = frame.xs('n', level='bz')['an'].values
    bar_g=ax2.barh(agents_ticks,g_calls,color=gcol,height=0.8,label='calls gesamt')
    bar_k=ax2.barh(agents_ticks,k_calls,color=kc_col,height=0.2,label='davon in Kernzeit')
    bar_n=ax2.barh(agents_ticks,n_calls,color=nc_col,left=k_calls,height=0.2,label='davon in Nebenzeit')
    
    ## X-Achse soll hier von rechts nach links gehen
    x2=ax2.get_xlim()    # normale x-achse
    x2_rev=(x2[1]+15,x2[0]) # koordinaten umgedreht + 10 Einheiten Puffer zum Rand
    ax2.set_xlim(x2_rev) # und als neue achse gesetzt
    
    ## Hilfslinien
    ax2.grid(which='major', axis='x', linestyle='-', zorder=0, alpha=1)
    ax2.grid(which='minor', axis='x', linestyle='--', zorder=0, alpha=0.5)
    ax2.set_axisbelow(True)
    
    ## Summe der Calls in ein Eck
    ax2.text(0.5, 0.005, 'Summe: '+str(summe_calls_gesamt), transform = ax2.transAxes, color="black", fontsize='10',horizontalalignment='center')
    
    #####################################################
    ## Einstellungen, die die ganze fig betreffen
    
    ## Abstand der Plots zueinander
    plt.subplots_adjust(wspace=0.4)
    
    ## Titel für die beiden Subplots jeweils nach oben
    ax1.set_xlabel('Minuten')
    ax1.xaxis.set_label_coords(0.5,1.05)
    ax2.set_xlabel('Calls')
    ax2.xaxis.set_label_coords(0.5,1.05)
    
    titel='Aufschlüsselung nach Agent für den Zeitraum '+vonbis[0].strftime('%D')+' bis '+vonbis[1].strftime('%D')
    fig.suptitle(titel,fontsize=9)
    
    ## Legende
    right_handles, right_labels = ax1.get_legend_handles_labels()
    left_handles, left_labels = ax2.get_legend_handles_labels()
    handles=[right_handles[3],left_handles[1],left_handles[2],right_handles[4],right_handles[5],right_handles[0],right_handles[1],right_handles[2]]
    labels=['Calls Kern+Neben | HT Kern+Neben',left_labels[1],left_labels[2],right_labels[4],right_labels[5],right_labels[0],right_labels[1],right_labels[2]]
    
    fig.legend(handles,labels,loc='lower right', fontsize=7, bbox_to_anchor=(0.95,0.15),ncol=1,framealpha=0.9)
    ax1.set_facecolor('#F1EFE7')
    ax2.set_facecolor('#E9F1E7')

    plotname='Agenten_'+vonbis[0].strftime('%Y_%m_%d')+'_bis_'+vonbis[1].strftime('%Y_%m_%d')
    fig.canvas.set_window_title(plotname)

## Daten und Dataframe

In [154]:
big_frame=pd.read_pickle(pklpth)  # wo kommt der pickle her und wie generiere ich einen aktualisierten? -> Datensammler.py
big_frame[['ht_float','tt_float']]=big_frame[['ht_float','tt_float']].apply(pd.to_numeric)
big_frame['acw_float']=big_frame['ht_float']-big_frame['tt_float']
big_frame['calweeks']=big_frame.index.strftime('%Y-%V')

#### von Hand

In [189]:
## Zeitraum für den plot wählen und filtern
#von_bis=(datetime.date(2018,4,1),datetime.date(2018,4,30))
#plotframe,von_bis_real=prepare_plotframe(big_frame,von_bis)
#agentplot(plotframe,von_bis_real)

#### mit loop mehrere Monate

In [208]:
plotyear=2018
for plotmonth in range(4,5):
    lastday=calendar.monthrange(plotyear,plotmonth)[1]
    von_bis=(datetime.date(plotyear,plotmonth,1),datetime.date(plotyear,plotmonth,lastday))
    plotframe,von_bis_real=prepare_plotframe(big_frame,von_bis)
    agentplot(plotframe,von_bis_real)
  
    